In [1]:
from rapidfuzz import process as pr
import pandas as pd

### RapidFuzz

In [2]:
# #load set of DW keywords before fuzzyWuzzy into the file
uni_kw=pd.read_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy.csv', names = ['ind', 'keyword'], header=0) 
#unique_keywords=uni_kw

In [3]:
print(len(uni_kw))
uni_kw.head()

32704


,ind,keyword
0,0,malware
1,1,state repression
2,2,climate consensus
3,3,Canary Islands
4,4,Angela I


In [4]:
ratio_array= pr.cdist(list(uni_kw['keyword']), list(uni_kw['keyword']), score_cutoff = 90)

In [5]:
len(ratio_array)
df_array = pd.DataFrame(ratio_array, columns = list(uni_kw['keyword']))
df_array.head(6)

,malware,state repression,climate consensus,Canary Islands,Angela I,Evan Rachel Wood,used cars,woodcarving,taboo,Grüne Woche,...,Drachen Lord,Missionaries,kiss-in,Fintech,Manuel Noriega,nuclear arms,processionary caterpillars,Deutsche Kammerphilharmonie Bremen,Maduro,self-sufficiency
0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
indices_correlating_rows = []
for i, row in enumerate(ratio_array):
    non_zero_values = [el for el in row if el!=0]
    if len(non_zero_values)>1:
        indices_correlating_rows.append(i)

In [ ]:
indices_correlating_rows